* Jiacheng(George) Zheng UID: 005856460
* Jiaxin(Frank) Zheng UID: 806310846
* Joy Lin UID: 806304435

### Question 1. 
* **Parameters**

\begin{align*}
i &: \text{hospital departments } \\
j &: \text{operating rooms } \\
k &: \text{days from Monday to Friday } \\
t_i &: \text{target hours for department } i \text{ based on the new 213.5 hours of operating room time per week} \\
h_{jk} &: \text{hours in operating room } j \text{ on day } k
\end{align*}

* **Decision Variables**

\begin{align*}
s_i &: \text{under-allocation of department } i \\
x_{ijk} &: \text{1 if department } i \text{ is assigned to operating room } j \text{ , on day } k \text{ , 0 otherwise }
\end{align*}

* **Objective Function**

\begin{equation*}
{Minimize} \quad \sum_{i=1}^{6} \frac{s_i}{t_i}
\end{equation*}

* **Constraints**

> Allocation Constraints (Ensuring target hours):

\begin{equation*}
\sum_{j=1}^{5}\sum_{k=1}^{5} x_{ijk} \cdot h_{jk} \geq t_i - s_i, \quad i = 1, 2, 3, 4, 5, 6
\end{equation*}

> Room Availability Constraints (One department per room per day):

\begin{equation*}
\sum_{i=1}^{6} x_{ijk} \leq 1, \quad j = \text{1, 2, 3, 4, 5 } \text{  } k = \text{1, 2, 3, 4, 5 }
\end{equation*}

> Total Hours Constraint (Total hours do not exceed 213.5 hours per week):

\begin{equation*}
\sum_{i=1}^{6}\sum_{j=1}^{5}\sum_{k=1}^{5} x_{ijk} \cdot h_{jk} \leq 213.5
\end{equation*}

> Non-Negative and Binary Constraints:

\begin{align*}
s_i &\geq 0, \quad i = 1, 2, 3, 4, 5, 6 \\
x_{ijk} &\in \{0, 1\}, \quad i = \text{1, 2, 3, 4, 5, 6 } \text{  } j = \text{1, 2, 3, 4, 5 } \text{  } k = \text{1, 2, 3, 4, 5 }
\end{align*}

In [22]:
from gurobipy import *

# Initialize the model
mod = Model("OperatingRoomScheduling")

# Constants and mappings
departments, rooms, days = 6, 5, 5
target_hours = [103.334, 8.967, 54.0155, 15.799, 11.3155, 20.2825]
room_hours = [[9, 9, 9, 9, 9], [9, 9, 9, 9, 8], [9, 9, 9, 9, 8], [9, 9, 9, 9, 8], [7.5, 7.5, 7.5, 7.5, 6.5]]
max_total_hours = 213.5

# Decision variables
x = mod.addVars(departments, rooms, days, vtype=GRB.BINARY)
s = mod.addVars(departments)

# Objective function: Minimize total under-allocation percentage
mod.setObjective(sum(s[i] / target_hours[i] for i in range(departments)), GRB.MINIMIZE)

# Constraints
for i in range(departments):
    # Allocation constraints
    mod.addConstr(sum(x[i, j, k] * room_hours[j][k] for j in range(rooms) for k in range(days)) >= target_hours[i] - s[i])

for i in range(departments):
    mod.addConstr(s[i] >= 0)
    
for j in range(rooms):
    for k in range(days):
        # Room availability constraints
        mod.addConstr(sum(x[i, j, k] for i in range(departments)) <= 1)

mod.addConstr(sum(x[i, j, k] * room_hours[j][k] for i in range(departments) for j in range(rooms) for k in range(days)) <= max_total_hours)

# Solve the model
mod.optimize()

# Display results
if mod.status == GRB.OPTIMAL:
    print("Optimal objective value:", mod.objval)
    for i in range(departments):  # Iterate over the range of departments
        for j in range(rooms):
            for k in range(days):
                if x[i, j, k].x > 0.5:
                    print(f"Department {i+1} is allocated Room {j+1} on Day {k+1}")
    
    # Corrected iteration over departments
    print(f"Optimal objective value: {mod.objVal}")
    for i in range(departments):  # Corrected: Use range(departments)
        allocated_hours = sum(x[i, j, k].x * room_hours[j][k] for j in range(rooms) for k in range(days))
        print(f"Department {i+1} allocated hours: {allocated_hours}")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 38 rows, 156 columns and 462 nonzeros
Model fingerprint: 0x749fcd6c
Variable types: 6 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective 6.0000000
Found heuristic solution: objective 6.0000000
Presolve removed 7 rows and 0 columns
Presolve time: 0.00s
Presolved: 31 rows, 156 columns, 306 nonzeros
Variable types: 4 continuous, 152 integer (150 binary)
Found heuristic solution: objective 5.0000000

Root relaxation: objective 2.066116e-03, 50 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |

### Question 2. 
* **Parameters**

\begin{align*}
i &: \text{hospital departments } \\
j &: \text{operating rooms } \\
k &: \text{days from Monday to Friday } \\
f &: \text{floor } \\
t_i &: \text{target hours for department } i \text{ based on the new 213.5 hours of operating room time per week} \\
h_{jk} &: \text{hours in operating room } j \text{ on day } k
\end{align*}

* **Decision Variables**

\begin{align*}
s_i &: \text{under-allocation of department } i \\
x_{ijk} &: \text{1 if department } i \text{ is assigned to operating room } j \text{ , on day } k \text{ , 0 otherwise } \\
y_{ikf} &: \text{1 if department } i \text{ is assigned to floor } f \text{ on day } k \text{ , 0 otherwise }
\end{align*}

* **Objective Function**

\begin{equation*}
{Minimize} \quad \sum_{i=1}^{6} \frac{s_i}{t_i}
\end{equation*}

* **Constraints**

> Allocation Constraints (Ensuring target hours):

\begin{equation*}
\sum_{j=1}^{5}\sum_{k=1}^{5} x_{ijk} \cdot h_{jk} \geq t_i - s_i, \quad i = 1, 2, 3, 4, 5, 6
\end{equation*}

> Room Availability Constraints (One department per room per day):

\begin{equation*}
\sum_{i=1}^{6} x_{ijk} \leq 1, \quad j = \text{1, 2, 3, 4, 5 } \text{  } k = \text{1, 2, 3, 4, 5 }
\end{equation*}

> Total Hours Constraint (Total hours do not exceed 213.5 hours per week):

\begin{equation*}
\sum_{i=1}^{6}\sum_{j=1}^{5}\sum_{k=1}^{5} x_{ijk} \cdot h_{jk} \leq 213.5
\end{equation*}


> Floor Assignment Constraint:

\begin{equation*}
\sum_{l=1}^{3} y_{ikf} \leq 1 \text{  } i = \text{1, 2, 3, 4, 5, 6 } \text{  } k = \text{1, 2, 3, 4, 5 } \text{  } f = \text{1, 2, 3 }
\end{equation*}

>Consistency Constraint:

\begin{equation*}
x_{ijk} \leq y_{ikl} \text{  } i = \text{1, 2, 3, 4, 5, 6 } \text{  } j = \text{1, 2, 3, 4, 5 } \text{  } k = \text{1, 2, 3, 4, 5 } \text{  } f = \text{1, 2, 3 }
\end{equation*}

> Non-Negative and Binary Constraints:

\begin{align*}
s_i &\geq 0, \quad i = 1, 2, 3, 4, 5, 6 \\
x_{ijk} &\in \{0, 1\}, \quad i = \text{1, 2, 3, 4, 5, 6 } \text{  } j = \text{1, 2, 3, 4, 5 } \text{  } k = \text{1, 2, 3, 4, 5 }
\end{align*}

In [23]:
from gurobipy import *

# Initialize the model
mod = Model("OperatingRoomScheduling")

# Constants and mappings
departments, rooms, days = 6, 5, 5
floors = [1, 2, 3]
target_hours = [103.334, 8.967, 54.0155, 15.799, 11.3155, 20.2825]
room_hours = [[9, 9, 9, 9, 9], [9, 9, 9, 9, 8], [9, 9, 9, 9, 8], [9, 9, 9, 9, 8], [7.5, 7.5, 7.5, 7.5, 6.5]]
room_floors = {0: 1, 1: 1, 2: 2, 3: 2, 4: 3}
max_total_hours = 213.5

# Decision variables
x = mod.addVars(departments, rooms, days, vtype=GRB.BINARY)
s = mod.addVars(departments, name="s")
y = mod.addVars(departments, days, floors, vtype=GRB.BINARY)

# Objective function: Minimize total under-allocation percentage
mod.setObjective(sum(s[i] / target_hours[i] for i in range(departments)), GRB.MINIMIZE)

# Constraints
for i in range(departments):
    # Allocation constraints
    mod.addConstr(sum(x[i, j, k] * room_hours[j][k] for j in range(rooms) for k in range(days)) >= target_hours[i] - s[i])
    
    for k in range(days):
        # Floor assignment constraints
        mod.addConstr(sum(y[i, k, f] for f in floors) <= 1)

        for j in range(rooms):
            # Consistency constraints
            mod.addConstr(x[i, j, k] <= y[i, k, room_floors[j]])

for i in range(departments):
    mod.addConstr(s[i] >= 0)

for j in range(rooms):
    for k in range(days):
        # Room availability constraints
        mod.addConstr(sum(x[i, j, k] for i in range(departments)) <= 1)

# Total Hours Constraint
mod.addConstr(sum(x[i, j, k] * room_hours[j][k] for i in range(departments) for j in range(rooms) for k in range(days)) <= max_total_hours)

# Solve the model
mod.optimize()

# Display results
if mod.status == GRB.OPTIMAL:
    print("Optimal objective value:", mod.objval)
    for i in range(departments):  # Iterate over the range of departments
        for j in range(rooms):
            for k in range(days):
                if x[i, j, k].x > 0.5:
                    print(f"Department {i+1} is allocated Room {j+1} on Day {k+1}")
    
    # Corrected iteration over departments
    print(f"Optimal objective value: {mod.objVal}")
    for i in range(departments):  # Corrected: Use range(departments)
        allocated_hours = sum(x[i, j, k].x * room_hours[j][k] for j in range(rooms) for k in range(days))
        print(f"Department {i+1} allocated hours: {allocated_hours}")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 218 rows, 246 columns and 852 nonzeros
Model fingerprint: 0x92663c29
Variable types: 6 continuous, 240 integer (240 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective 6.0000000
Found heuristic solution: objective 6.0000000
Presolve removed 67 rows and 60 columns
Presolve time: 0.00s
Presolved: 151 rows, 186 columns, 606 nonzeros
Variable types: 4 continuous, 182 integer (180 binary)
Found heuristic solution: objective 5.9998574

Root relaxation: objective 1.387152e-01, 97 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds   